2.1 - 2.2 Migration: pblum_mode and pblum vs pblum_ext
==============================

PHOEBE 2.2 introduces new modes for handling the scaling between absolute and relative luminosities/intensities/fluxes via the new `pblum_mode` parameter, which will exist for each LC dataset attached to the bundle.  By default `pblum_mode` will be set to 'component-coupled', which mimics the default behavior prior to version 2.2.

In [1]:
import phoebe
b = phoebe.default_binary()
b.add_dataset('lc', dataset='lc01')
print(b.filter(qualifier='pblum*', dataset='lc01'))

ParameterSet: 3 parameters
         pblum_mode@lc01@dataset: component-coupled
    pblum_component@lc01@dataset: primary
      pblum@primary@lc01@dataset: 12.566370614359172 W


In [2]:
print(b.get_parameter('pblum_mode'))

Parameter: pblum_mode@lc01@dataset
                       Qualifier: pblum_mode
                     Description: Mode for scaling passband luminosities
                           Value: component-coupled
                         Choices: decoupled, component-coupled, dataset-coupled, dataset-scaled, absolute
                  Constrained by: 
                      Constrains: None
                      Related to: None



In the default mode, you can change which of the stars you'd like to provide the luminosity.  By default, this is the primary component.  To provide the luminosity of the secondary star *instead*, set `pblum_component`.  

Previously this was achieved by setting `pblum_ref@primary = 'secondary'` and `pblum_ref@secondary = 'self'`.  Note that the `pblum_ref` parameter has been removed in 2.2+ in favor of the more flexible and intuitive `pblum_mode` parameter.

In [3]:
print(b.get_parameter('pblum_component'))

Parameter: pblum_component@lc01@dataset
                       Qualifier: pblum_component
                     Description: Which component's pblum will be provided
                           Value: primary
                         Choices: primary, secondary
                  Constrained by: 
                      Constrains: None
                      Related to: None
                 Only visible if: pblum_mode:component-coupled



In [4]:
b.set_value('pblum_component', 'secondary')
print(b.filter(qualifier='pblum*', dataset='lc01'))

ParameterSet: 3 parameters
         pblum_mode@lc01@dataset: component-coupled
    pblum_component@lc01@dataset: secondary
    pblum@secondary@lc01@dataset: 12.566370614359172 W


Previously, you could 'decouple' the luminisoties by setting `pblum_ref` of both components to 'self'.  In PHOEBE 2.2+, you will instead change `pblum_mode` to 'decoupled', in which case multiple `pblum` parameters will become visible.

In [5]:
b.set_value('pblum_mode', 'decoupled')
print(b.filter(qualifier='pblum*', dataset='lc01'))

ParameterSet: 3 parameters
         pblum_mode@lc01@dataset: decoupled
      pblum@primary@lc01@dataset: 12.566370614359172 W
    pblum@secondary@lc01@dataset: 12.566370614359172 W


For more information on the behavior for all of these supported modes, see the [pblum tutorial](./pblum.ipynb).

In addition, PHOEBE 2.2 distinguishes between intrinsic passband luminosities (`pblum`) and extrinsic passband luminosities (`pblum_ext`).  The returned dictionary from [b.compute_pblums](../api/phoebe.frontend.bundle.Bundle.compute_pblums.md) now includes both intrinsic and extrinsic values, with the keys of the dictionary now including `pblum@` or `pblum_ext@`.

In [6]:
print(b.compute_pblums())

{'pblum_ext@primary@lc01': <Quantity 12.64179719 W>, 'pblum_ext@secondary@lc01': <Quantity 12.64179719 W>, 'pblum@primary@lc01': <Quantity 12.56637061 W>, 'pblum@secondary@lc01': <Quantity 12.56637061 W>}


This also means that the mesh column to expose luminosities is renamed to `pblum_ext` (and `abs_pblum_ext`) as these expose the extrinsic luminosities (including features such as spots, irradiation, etc).

In [7]:
b.add_dataset('mesh', dataset='mesh01')

<ParameterSet: 8 parameters | contexts: constraint, dataset, compute>

In [8]:
print(b.get_parameter('columns', dataset='mesh01').choices)

['volume', 'xs', 'ys', 'zs', 'vxs', 'vys', 'vzs', 'nxs', 'nys', 'nzs', 'us', 'vs', 'ws', 'vus', 'vvs', 'vws', 'nus', 'nvs', 'nws', 'areas', 'loggs', 'teffs', 'rprojs', 'mus', 'visibilities', 'visible_centroids', 'rs', 'intensities@lc01', 'normal_intensities@lc01', 'abs_intensities@lc01', 'abs_normal_intensities@lc01', 'boost_factors@lc01', 'ldint@lc01', 'pblum_ext@lc01', 'abs_pblum_ext@lc01', 'ptfarea@lc01']
